In [4]:
import datetime
import os
import time
import random

import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Activation, BatchNormalization, Dense,
                                     Dropout, Flatten, Input, Reshape,
                                     TimeDistributed)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop

tf.compat.v1.disable_eager_execution()

In [5]:
%reload_ext tensorboard

In [3]:
import midi

In [4]:
NUM_EPOCHS = 100
LR = 0.001
WRITE_HISTORY = True
NUM_RAND_SONGS = 10
DO_RATE = 0.1
BN_M = 0.9

BATCH_SIZE = 128
MAX_LENGTH = 16
PARAM_SIZE = 120

np.random.seed(0)
random.seed(0)

In [16]:
y_samples = np.load('samples.npy')
y_lengths = np.load('lengths.npy')
# y_samples = np.load('samplesBach.npy')
# y_lengths = np.load('lengthsBach.npy')
num_songs = y_lengths.shape[0]

y_shape = (num_songs, MAX_LENGTH) + y_samples.shape[1:]
y_orig = np.zeros(y_shape, dtype=y_samples.dtype)

In [17]:
cur_ix = 0
for i in range(num_songs):
    end_ix = cur_ix + y_lengths[i]
    for j in range(MAX_LENGTH):
        k = j % (end_ix - cur_ix) 
        y_orig[i,j] = y_samples[cur_ix + k]
    cur_ix = end_ix

In [18]:
y = np.copy(y_orig)

y_train = y[:175]
y_valid = y[175:]

y_test_song = np.copy(y[0])
midi.samples_to_midi(y_test_song, 'gt.mid', 16)

In [19]:
x_in = Input(shape=y_shape[1:])
x = Reshape((y_shape[1], -1))(x_in)
x = TimeDistributed(Dense(2000, activation='relu'))(x)
x = TimeDistributed(Dense(200, activation='relu'))(x)
x = Flatten()(x)
x = Dense(1600, activation='relu')(x)
x = Dense(PARAM_SIZE)(x)
x = BatchNormalization(momentum=BN_M, name='pre_encoder')(x)

x = Dense(1600, name='encoder')(x)
x = BatchNormalization(momentum=BN_M)(x)
x = Activation('relu')(x)
x = Dropout(DO_RATE)(x)
x = Dense(MAX_LENGTH * 200)(x)
x = Reshape((MAX_LENGTH, 200))(x)
x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
x = Activation('relu')(x)
x = Dropout(DO_RATE)(x)
x = TimeDistributed(Dense(2000))(x)
x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
x = Activation('relu')(x)
x = Dropout(DO_RATE)(x)
x = TimeDistributed(Dense(y_shape[2] * y_shape[3], activation='sigmoid'))(x)
x = Reshape((y_shape[1], y_shape[2], y_shape[3]))(x)

model = Model(x_in, x)
model.compile(optimizer=RMSprop(learning_rate=LR), loss='binary_crossentropy')

In [20]:
func = K.function([model.get_layer('encoder').input, K.learning_phase()], [model.layers[-1].output])
enc = Model(inputs=model.input, outputs=model.get_layer('pre_encoder').output)

In [21]:
rand_vecs = np.random.normal(0.0, 1.0, (NUM_RAND_SONGS, PARAM_SIZE))
np.save('rand.npy', rand_vecs)

In [22]:
log_dir = os.path.join(
    "logs",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=log_dir),
]

In [23]:
history = model.fit(
    y_train,
    y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(y_valid, y_valid),
    callbacks=callbacks,
    verbose=2,
)

loss = history.history["loss"][-1]
print(f"Train Loss: {loss}")

write_dir = 'HistoryAuto/'
model.save('HistoryAuto/model.h5')

Train on 175 samples, validate on 63 samples
Epoch 1/100


C:\Users\eller\anaconda3\envs\IC\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


175/175 - 9s - loss: 0.5661 - val_loss: 0.6461
Epoch 2/100
175/175 - 4s - loss: 0.0753 - val_loss: 0.4836
Epoch 3/100
175/175 - 4s - loss: 0.0456 - val_loss: 0.4178
Epoch 4/100
175/175 - 4s - loss: 0.0313 - val_loss: 0.3320
Epoch 5/100
175/175 - 4s - loss: 0.0262 - val_loss: 0.2488
Epoch 6/100
175/175 - 4s - loss: 0.0239 - val_loss: 0.1741
Epoch 7/100
175/175 - 4s - loss: 0.0223 - val_loss: 0.1387
Epoch 8/100
175/175 - 4s - loss: 0.0212 - val_loss: 0.0978
Epoch 9/100
175/175 - 4s - loss: 0.0204 - val_loss: 0.0681
Epoch 10/100
175/175 - 4s - loss: 0.0197 - val_loss: 0.0652
Epoch 11/100
175/175 - 4s - loss: 0.0192 - val_loss: 0.0496
Epoch 12/100
175/175 - 4s - loss: 0.0188 - val_loss: 0.0327
Epoch 13/100
175/175 - 4s - loss: 0.0185 - val_loss: 0.0359
Epoch 14/100
175/175 - 4s - loss: 0.0183 - val_loss: 0.0284
Epoch 15/100
175/175 - 4s - loss: 0.0179 - val_loss: 0.0236
Epoch 16/100
175/175 - 4s - loss: 0.0178 - val_loss: 0.0223
Epoch 17/100
175/175 - 4s - loss: 0.0176 - val_loss: 0.0225
E

In [24]:
def make_rand_songs(write_dir, rand_vecs):
    for i in range(rand_vecs.shape[0]):
        x_rand = rand_vecs[i:i+1]
        y_song = func([x_rand, 0])[0]
        midi.samples_to_midi(y_song[0], write_dir + 'randGame' + str(i) + '.mid', 16, 0.5)

def make_rand_songs_normalized(write_dir, rand_vecs):
    x_enc = np.squeeze(enc.predict(y_orig))

    x_mean = np.mean(x_enc, axis=0)
    x_cov = np.cov((x_enc - x_mean).T)
    _, s, v = np.linalg.svd(x_cov)
    e = np.sqrt(s)

    print(f"Means: {x_mean[:6]}")
    print(f"Evals: {e[:6]} ")

    x_vecs = x_mean + np.dot(rand_vecs * e, v)
    make_rand_songs(write_dir, x_vecs)

In [25]:
y_song = model.predict((y_test_song).reshape(1,16,96,96), batch_size=BATCH_SIZE)

midi.samples_to_midi(y_song[0], write_dir + 'test.mid', 16)
make_rand_songs_normalized(write_dir, rand_vecs)

Means: [0.02104055 0.17722142 0.09916551 0.16378419 0.02010539 0.062884  ]
Evals: [4.58530266 3.55321573 3.27695911 3.12697031 2.93756239 2.77289109] 
